# [ Parallel Corpus: 병렬 코퍼스 ]

*   corpus(말뭉치 또는 코퍼스)란 언어의 표본을 추출한 집합으로, 보통 텍스트 집합을 의미한다.
*   parallel corpus는 2개 이상의 언어가 병렬적으로 구성된 것을 의미한다.
*   시퀀스-투-시퀀스 에서는 기본적으로 입력 시퀀스와 출력 시퀀스의 길이가 다를 수 있다고 가정한다.



# [ 데이터 설명 ]
(다운로드 링크) [http://www.manythings.org/anki](http://www.manythings.org/anki)

*   **src** : (source의 줄임말) 입력 문장을 나타낸다.
*   **tar** : (target의 줄임말) 번역하고자 하는 문장을 나타낸다.
*   **fra.txt** : 아래와 같이 영어 문장과 프랑스어 문장 사이에 탭으로 구분되는 구조가 하나의 샘플 + 약 16만 개의 병렬 문장 샘플을 포함

         Watch me.           Regardez-moi !




------




In [ ]:
import pandas as pd
import urllib3
import zipfile
import shutil
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
http=urllib3.PoolManager()
url='http://www.manythings.org/anki/fra-eng.zip'
filename='fra-eng.zip'

path=os.getcwd() 
zipfilename=os.path.join(path, filename)

with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:
  shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
  zip_ref.extractall(path)

#[ 모듈 & 코드 설명 ]

*   **urllib** : URL 작업을 위한 여러 모듈을 모은 패키지이다.
*  **urllib3** : powerful HTTP client for Python.
*   **OS 모듈** : 환경 변수나 디렉터리, 파일과 같은 OS 자원을 제어 할 수 있게 해주는 모듈이다.
*   **zipfile 모듈** : 개별 또는 여러 파일을 압축하거나 압축 해제 해주는 모듈이다.
  *   os.getcwd( ) : 현재 자신의 디렉터리 위치를 반환해준다.
  *   open( ) : 파일을 읽기 위해서는 'rb'로, 쓰기 위해서는 'wb'로 파일 모드를 지정한다.
  *   zipfile.Zipfile( ) : 해당 파일을 읽기 권한('r')으로 오픈한다.
  *   extractall( ) : 해당 경로에 파일을 압축 해제 해준다.




# [ HTTP 프로토콜]


*  프로토콜이란, 상호 간에 정의한 규칙을 의미하며 특정 기기 간에 데이터를 주고 받기 위해 정의 되었다.
*   웹에서는 브라우저와 서버 간에 데이터를 주고 받기 위해서 HTTP 프로토콜을 사용한다.
*   HTTP 프로토콜은 stateless 프로토콜이므로, 이전 데이터 요청과 다음 데이터 요청이 서로 관련이 없다. 
*   HTTP 프로토콜은 일반적으로 TCP/IP 통신 위에서 동작한다.
*   HTTP 요청 메서드(Http Request Methods) : URL을 이용하여 요청한 서버의 데이터에 특정 동착을 수행하고 싶을 때 사용한다.

         * GET : 존재하는 자원에 대한 요청
         * POST : 새로운 자원을 생성
         * PUT : 존재하는 자원에 대한 변경
         * DELETE : 존재하는 자원에 대한 삭제




------





In [ ]:
lines= pd.read_csv('fra.txt', names=['src', 'tar', 'etc'], sep='\t')
len(lines)

177210



*   전체 샘플은 177210개 존재한다.



In [ ]:
lines

,src,tar,etc
0,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
2,Hi.,Salut.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
3,Run!,Cours !,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
4,Run!,Courez !,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
...,...,...,...
177205,"Top-down economics never works, said Obama. ""T...","« L'économie en partant du haut vers le bas, ç...",CC-BY 2.0 (France) Attribution: tatoeba.org #1...
177206,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
177207,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
177208,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


In [ ]:
lines=lines.loc[:, 'src':'tar']  # 전체 샘플에 대해서 'src' 레이블(label) 부터 'tar' 레이블(label) 까지 선택한다.
lines=lines[0:60000]   # 6만 개의 샘플만 가지고 기계 번역기를 구축한다.
lines.sample(10)       # 랜덤으로 선택된 10개의 샘플을 통해, 현재 데이터가 어떻게 구성되어 있는지 확인해본다.

,src,tar
3638,I handled it.,J'ai su y faire.
29450,I don't need it now.,Je n'en ai pas besoin à l'heure actuelle.
49192,I'm allowing you to go.,Je t'autorise à y aller.
49967,Look at the blackboard.,Regardez le tableau.
24655,I want to tell you.,Je veux vous le dire.
40028,Are you in a bad mood?,Es-tu de mauvaise humeur ?
42233,I ran away in a hurry.,Je suis parti en courant précipitamment.
40376,Do you have some milk?,Avez-vous un peu de lait ?
44251,That's the difference.,C'est la différence.
38439,Tom is heavily armed.,Tom est lourdement armé.


In [ ]:
lines.tar=lines.tar.apply(lambda x : '\t'+x+'\n')   #lambda 매개변수 : 반환값으로 사용할 식 ← lambda 를 이용해서 익명 함수를 호출하려면 변수(lines.tar)를 할당해줘야한다.
lines.sample(10)

,src,tar
47137,Everybody calls me Tom.,\tTout le monde m'appelle Tom.\n
27445,Who's your teacher?,\tQui est votre professeur ?\n
1669,Taste this.,\tGoûtez ceci.\n
31473,Stop being so naive.,\tArrêtez d'être si naïve.\n
44935,Tom is in the kitchen.,\tTom est dans la cuisine.\n
17156,Were you excited?,\tÉtais-tu excité ?\n
16763,Tom is in church.,\tTom est à l'église.\n
26630,Tom has green eyes.,\tTom a les yeux verts.\n
25833,She shook her head.,\tElle a secoué la tête.\n
53069,A horse passed my house.,\tUn cheval passa devant chez moi.\n




*   (tar: 번역하고자 하는 문장에) 기계 번역을 위해서 시작을 의미하는 심볼 ' \t ' 와 종료를 의미하는 심볼 ' \n ' 을 추가해주었다.



In [ ]:
# 글자 집합 구축하기
src_vocabulary=set()
for line in lines.src:  # src열의 한 행씩 읽는다.
  for char in line:     # 해당 문장(행)에서 1개의 글자(문자)씩 읽는다.
    src_vocabulary.add(char)

tar_vocabulary=set()
for line in lines.tar:
  for char in line:
    tar_vocabulary.add(char)



# [ 집합자료형(set)의 특징 ]
 *  중복을 허용하지 않는다.
 *  순서가 존재하지 않는다.
 *  set 자료형은 순서가 없기 때문에 인덱싱으로 값을 얻을 수 없다.
 *  set 자료형을 인덱싱으로 접근하려면, 아래와 같이 리스트나 튜플로 변환해서 사용해야 한다.

            set1 = set([1,2,3])
            list1 = list(set1)
            tuple = tuple(set1)




In [ ]:
# 집합자료형을 리스트로 변환해서 정렬해준다.
src_vocabulary = sorted(list(src_vocabulary))
tar_vocabulary = sorted(list(tar_vocabulary))

# 정렬해준 자료형에 각각 인덱스를 부여해준다.
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocabulary)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocabulary)])

print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, '’': 77, '€': 78}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, ',': 11, '-': 12, '.': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, '?': 25, 'A': 26, 'B': 27, 'C': 28, 'D': 29, 'E': 30, 'F': 31, 'G': 32, 'H': 33, 'I': 34, 'J': 3



---
# [ src(영어), tar(프랑스어) 각 문장의 글자(문자)에 대해서 정수 인코딩 ]


In [ ]:
encoder_input=[]
for line in lines.src:  # src열의 한 행씩 읽는다.
    temp_X=[]
    for w in line:      # 해당 문장(행)에서 1개의 글자(문자)씩 읽는다.
      temp_X.append(src_to_index[w])   # 글자를 해당되는 정수로 변환한다.
    encoder_input.append(temp_X)

In [ ]:
print(encoder_input[:5])
print(lines.src[:5])

[[30, 64, 10], [31, 58, 10], [31, 58, 10], [41, 70, 63, 2], [41, 70, 63, 2]]
0     Go.
1     Hi.
2     Hi.
3    Run!
4    Run!
Name: src, dtype: object




*   영어 문장 샘플에 대해서 정수 인코딩이 되었음을 확인 할 수 있다.
*   G (30) / o(64) / .(10)



In [ ]:
# 예측에 사용될 정수 인코딩

decoder_input=[]
for line in lines.tar:
  temp_X=[]
  for w in line:
    temp_X.append(tar_to_index[w])
  decoder_input.append(temp_X)

In [ ]:
print(decoder_input[:5])
print(lines.tar[:5])

[[1, 47, 52, 3, 4, 2], [1, 44, 52, 63, 72, 71, 3, 4, 2], [1, 44, 52, 63, 72, 71, 13, 2], [1, 28, 66, 72, 69, 70, 104, 4, 2], [1, 28, 66, 72, 69, 56, 77, 104, 4, 2]]
0        \tVa !\n
1     \tSalut !\n
2      \tSalut.\n
3     \tCours !\n
4    \tCourez !\n
Name: tar, dtype: object


In [ ]:
# 실제 정수 인코딩(시작 심볼 '\t'이 존재할 필요가 없으므로 제거해준다.)

decoder_target=[]
for line in lines.tar:
  t=0
  temp_X=[]
  for w in line:
    if t>0:
      temp_X.append(tar_to_index[w])
    t=t+1
  decoder_target.append(temp_X)

In [ ]:
print(decoder_target[:5])

[[47, 52, 3, 4, 2], [44, 52, 63, 72, 71, 3, 4, 2], [44, 52, 63, 72, 71, 13, 2], [28, 66, 72, 69, 70, 104, 4, 2], [28, 66, 72, 69, 56, 77, 104, 4, 2]]




---
# [ 패딩 작업과 원-핫 인코딩 변환 ]


*   영어와 프랑스어의 쌍으로 구성되어 있지만 길이가 모두 다르므로, 패딩을 할 때에 길이를 전부 동일하게 맞춰줄 필요는 없다.
*   영어 데이터는 영어 샘플들 끼리, 프랑스어는 프랑스어 샘플들끼리 길이를 맞추어서 패딩을 해준다.
*   아래의 실행 결과에 따라 영어 데이터의 샘플은 전부 길이가 25 가 되도록, 프랑스어는 74 가 되도록 패딩한다.




In [ ]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print(max_src_len)
print(max_tar_len)

25
74


In [ ]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [ ]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)